In [42]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import missingno as msn

import glob

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.float_format', '{:.2f}'.format)
# pd.set_option('display.width', None)


# Creating pickle 2018 - 2020

In [81]:
cols = ['yearly_budget', 'yearly_forecast', 'wlc_baseline_incl_NCG']

def load_and_clean_data(file_path):
    df = pd.read_csv(file_path, thousands=',', encoding="latin2")
    cols_to_clean = cols
    
    for col in cols_to_clean:
        df[col] = df[col].str.replace('Ł', '', regex=False).str.replace(',', '', regex=False)
    
    return df

# Load and clean data for each year
df_2018 = load_and_clean_data("../data/raw_data/uk_2018.csv")
df_2019 = load_and_clean_data("../data/raw_data/uk_2019.csv")
df_2020 = load_and_clean_data("../data/raw_data/uk_2020.csv")

# Concatenate dataframes
df_18_20 = pd.concat([df_2018, df_2019, df_2020], axis=0)

df_18_20["colour_rating"] = df_18_20["colour_rating"].str.lower()

# Transform datatype
for col in cols:
    df_18_20[col] = pd.to_numeric(df_18_20[col], errors='coerce')
    
df_18_20["start_date"] = pd.to_datetime(df_18_20["start_date"], errors="coerce", format="%d.%m.%y")
df_18_20["end_date"] = pd.to_datetime(df_18_20["end_date"], errors="coerce", format="%d.%m.%y")

# Cleaning colour rating and making exempts into NaNs
colour_rating = ['amber', 'green', 'red', 'amber/red', 'amber/green']
df_18_20["colour_rating"] = df_18_20["colour_rating"].str.lower()
df_18_20["colour_rating"] = np.where(df_18_20["colour_rating"].isin(colour_rating), df_18_20["colour_rating"], np.nan)

df_18_20.to_pickle("../data/pickle/cleaned_2018_2020.pkl")


In [80]:
df_18_20.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391 entries, 0 to 124
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   project_name           391 non-null    object        
 1   department             391 non-null    object        
 2   colour_rating          382 non-null    object        
 3   description_aims       391 non-null    object        
 4   rating_comment         391 non-null    object        
 5   start_date             384 non-null    datetime64[ns]
 6   end_date               375 non-null    datetime64[ns]
 7   schedule_comment       389 non-null    object        
 8   yearly_budget          361 non-null    float64       
 9   yearly_forecast        361 non-null    float64       
 10  wlc_baseline_incl_NCG  349 non-null    float64       
 11  variance_comment       390 non-null    object        
 12  budget_comment         391 non-null    object        
 13  year      

,project_name,department,colour_rating,description_aims,rating_comment,start_date,end_date,schedule_comment,yearly_budget,yearly_forecast,wlc_baseline_incl_NCG,variance_comment,budget_comment,year,report_category,project_number
0,Geological Disposal Facility Programme (GDF),BEIS,amber/red,The primary objective of the programme is to s...,The rating reflects the early stages of a long...,2008-06-30,2040-12-31,Planning schedules are in line with the ambiti...,32.24,32.25,12131.50,Budget variance less than 5%,Whole Life Cost figure represents the estimate...,2018,NaN,NaN
1,Heat Networks Investment Project,BEIS,NaN,The Heat Network Investment Project (HNIP) is ...,HNIP has undertaken detailed project planning ...,2015-11-25,2021-03-31,HNIP main scheme schedule is on track to revis...,33.67,21.60,365.98,Budget is underspent due to main scheme set up...,"As part of the 2015 Spending Review agreement,...",2018,NaN,NaN
2,Local Land Charges (LLC) Programme,BEIS,amber,A Local Land Charge (LLC) is a restriction or ...,In line with the newly approved Phase1 approac...,2014-03-01,2023-11-17,"Programme has been re-planned in January 2018,...",25.80,13.60,193.30,The in year underspends reflect the delays to ...,"Following approval of phase one activities, th...",2018,NaN,NaN
3,Magnox & RSRL PBO Competition,BEIS,amber,The objective of the procurement (the Magnox C...,"The Secretary of State for Business, Energy an...",2012-04-03,2015-08-31,The current contract with CFP will terminate a...,572.00,555.00,3081.00,Budget variance less than 5%,The Whole Life Cost covers the 5 year contract...,2018,NaN,NaN
4,New Polar Research Vessel,BEIS,amber/green,NERC/British Antarctic Survey has a business n...,NERCs Programme Office has been monitoring all...,2014-05-01,2022-12-31,Programme is on track to deliver to its agreed...,113.00,111.00,1403.00,Budget variance less than 5%,The whole life cost represents costs until 204...,2018,NaN,NaN
5,Sellafield Model Change (SMC),BEIS,amber/green,Changing the model for engaging the private se...,This complex programme of work is progressing ...,2015-01-13,2017-05-24,"In 2014, the original Sellafield Model Change ...",2341.70,2260.70,30011.20,Budget variance less than 5%,The budgeted whole life costs relate to the wh...,2018,NaN,NaN
6,Smart Metering Implementation Programme,BEIS,amber,The Government's vision is for every home in G...,The most recent IPA review assessed the Progra...,2009-12-02,2020-12-31,On schedule for 2020 subject to energy supplie...,736.51,737.76,17215.78,Budget variance less than 5%,The total budget whole life cost figures in th...,2018,NaN,NaN
7,UKRI Implementation Programme,BEIS,amber,The UKRI Implementation Programme will create ...,"Success framework is being developed, organisa...",2016-01-01,2018-10-01,Schedule on track.,9.40,9.40,9.40,Budget variance less than 5%,The programme recognised there was potential ...,2018,NaN,NaN
8,1617 New Property Model Programme,CO,amber/red,The GPA will transform the way property is man...,Significant progress made by Shadow GPA since ...,2015-04-01,2021-04-01,The NPM Programme remains on schedule to launc...,11.10,11.10,93.93,Budget variance less than 5%,The Whole Life Cost figures reported by NPM re...,2018,NaN,NaN
9,Commercial Capability Programme,CO,amber,The Commercial Capability Programme is deliver...,Over 17/18 the Commercial Capability Programme...,2015-01-01,2018-03-31,On track to deliver key enablers within scope ...,9.20,9.20,20.60,Budget variance less than 5%,Increased budget and costs in 17/18 to support...,2018,NaN,NaN


In [72]:
df_18_20["start_date"] = pd.to_datetime(df_18_20["start_date"], errors="coerce")
df_18_20["end_date"] = pd.to_datetime(df_18_20["end_date"], errors="coerce")

/var/folders/70/4jr84ymn1ld27cfhlyc5g5yc0000gn/T/ipykernel_74615/3526477870.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_18_20["end_date"] = pd.to_datetime(df_18_20["end_date"], errors="coerce")


In [71]:
df_18_20[df_18_20["end_date_new"].isna()]

,project_name,department,colour_rating,description_aims,rating_comment,start_date,end_date,schedule_comment,yearly_budget,yearly_forecast,wlc_baseline_incl_NCG,variance_comment,budget_comment,year,report_category,project_number,colour_rating_new,start_date_new,end_date_new
95,DREADNOUGHT,MOD,amber/red,"The design, development and manufacture of 4 D...",The standing up of the Defence Nuclear Organis...,2011-04-14,Exempt under Section 35 of the Freedom of Info...,Delivery Phase 2 (DP2) is the phase in which t...,1004.66,1249.39,31497.93,Forecast includes additional Ł233M uplift agre...,The 2015 Strategic Defence and Security Review...,2018,NaN,NaN,amber/red,2011-04-14,NaT
99,Joint Crypt Key Programme,MOD,NaN,To support the Departments Information Assuran...,Exempt under Section 26 of Freedom of Informat...,NaT,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,NaN,NaN,NaN,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,2018,NaN,NaN,NaN,NaT,NaT
106,Mechanised Infantry Programme,MOD,amber,Key to the STRIKE Concept and Mechanised Infan...,Exempt under Section 43 of the Freedom of Info...,NaT,Exempt under Section 43 of the Freedom of Info...,Exempt under Section 43 of the Freedom of Info...,NaN,NaN,NaN,Exempt under Section 43 of the Freedom of Info...,Exempt under Section 43 of the Freedom of Info...,2018,NaN,NaN,amber,NaT,NaT
107,MODnet Evolve,MOD,amber,The overarching objective of the programme is ...,Actions in progress.,2016-02-29,TBD,The original aim was to start procurement of t...,15.20,14.87,NaN,Budget variance less than 5%,The Whole Life Costs are being developed in li...,2018,NaN,NaN,amber,2016-02-29,NaT
116,Type 31e,MOD,NaN,The T31e programme aims to deliver a pipeline ...,Exempt under Section 43 of the Freedom of Info...,2016-01-04,Exempt under Section 43 of the Freedom of Info...,Exempt under Section 43 of the Freedom of Info...,NaN,NaN,NaN,Budget variance less than 5%,Exempt under Section 43 of the Freedom of Info...,2018,NaN,NaN,NaN,2016-01-04,NaT
25,Social Work England,DFE,amber,"Governments aim is to promote a strong, consi...",Substantial progress has been made to support ...,2016-09-05,Exempt under Section 22 of the Freedom of Info...,A majority of milestones are steady state and/...,11.35,7.89,21.31,The 2018-19 forecast is around 31% below the o...,The budgeted whole life costs are based on the...,2019,Government Transformation and Service Delivery,NaN,amber,2016-09-05,NaT
31,Crossrail Programme,DFT,red,A new high-frequency rail service which will i...,In August 2018 Crossrail Limited (CRL) announc...,2008-07-22,Not provided,"In August 2018 Crossrail Limited (CRL), a whol...",466.75,1089.20,15474.61,This constitutes the funding for the project a...,The original anticipated cost was previously e...,2019,Infrastructure and Construction,NaN,red,2008-07-22,NaT
98,DREADNOUGHT,MOD,amber,"The design, development and manufacture of 4 D...",The programme has continued to gather pace in...,2011-04-14,Exempt under Section 35 of the Freedom of Info...,The priority remains to deliver the build sche...,1505.63,1339.29,30060.07,Reduction in forecast driven by accounting tre...,The whole life costs is for the acquisition o...,2019,Military Capability,NaN,amber,2011-04-14,NaT
102,Joint Crypt Key Programme,MOD,NaN,To support the Departments Information Assuran...,Exempt under Section 26 of Freedom of Informat...,NaT,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,NaN,NaN,NaN,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,2019,Military Capability,NaN,NaN,NaT,NaT
108,Mechanised Infantry Programme,MOD,amber,Key to the STRIKE Concept and Mechanised Infan...,Exempt under Section 43 of the Freedom of Info...,NaT,Exempt under Section 43 of the Freedom of Info...,Exempt under Section 43 of the Freedom of Info...,NaN,NaN,NaN,Exempt unde

In [46]:
df_18_20["colour_rating"].unique()

array(['Amber/Red',
       'Exempt under Section 43 of the Freedom of Information Act 2000 (Commercial Interests)',
       'Amber', 'Amber/Green', 'Green', 'Red',
       'Exempt under Section 26 of Freedom of Information Act 2000 (Defence)',
       'Amber/red',
       'Exempt under sections 24 & 26 of the Freedom of Information Act (2000)',
       'Exempt under Section 24 of the Freedom of Information Act 2000 (National Security)',
       'Exempt under Section 24 of Freedom of Information Act 2000 (National Security)'],
      dtype=object)

In [63]:
df_18_20[df_18_20["colour_rating_new"].isna()]

,project_name,department,colour_rating,description_aims,rating_comment,start_date,end_date,schedule_comment,yearly_budget,yearly_forecast,wlc_baseline_incl_NCG,variance_comment,budget_comment,year,report_category,project_number,colour_rating_new
1,Heat Networks Investment Project,BEIS,exempt under section 43 of the freedom of info...,The Heat Network Investment Project (HNIP) is ...,HNIP has undertaken detailed project planning ...,25.11.15,31.03.21,HNIP main scheme schedule is on track to revis...,33.67,21.60,365.98,Budget is underspent due to main scheme set up...,"As part of the 2015 Spending Review agreement,...",2018,NaN,NaN,NaN
99,Joint Crypt Key Programme,MOD,exempt under section 26 of freedom of informat...,To support the Departments Information Assuran...,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,NaN,NaN,NaN,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,2018,NaN,NaN,NaN
110,Nuclear Warhead Capability Sustainment Programme,MOD,exempt under sections 24 & 26 of the freedom o...,1. Support in-service warhead and production ...,Exempt under Section 24 of Freedom of Informat...,01.04.08,30.04.25,Exempt under Section 24 of Freedom of Informat...,1013.69,1024.53,19894.38,Exempt under Section 27 of the Freedom of Info...,Exempt under Section 27 of the Freedom of Info...,2018,NaN,NaN,NaN
116,Type 31e,MOD,exempt under section 43 of the freedom of info...,The T31e programme aims to deliver a pipeline ...,Exempt under Section 43 of the Freedom of Info...,01.04.16,Exempt under Section 43 of the Freedom of Info...,Exempt under Section 43 of the Freedom of Info...,NaN,NaN,NaN,Budget variance less than 5%,Exempt under Section 43 of the Freedom of Info...,2018,NaN,NaN,NaN
102,Joint Crypt Key Programme,MOD,exempt under section 26 of freedom of informat...,To support the Departments Information Assuran...,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,NaN,NaN,NaN,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,2019,Military Capability,NaN,NaN
112,Nuclear Warhead Capability Sustainment Programme,MOD,exempt under section 24 of the freedom of info...,1. Support in-service warhead and production ...,Exempt under Section 24 of Freedom of Informat...,01.04.08,30.04.25,Exempt under Section 24 of Freedom of Informat...,1065.18,1039.76,19895.89,Budget variance less than 5%,Exempt under Section 27 of the Freedom of Info...,2019,Military Capability,NaN,NaN
117,Type 31e,MOD,exempt under section 43 of the freedom of info...,The T31e programme aims to deliver a pipeline ...,Exempt under Section 43 of the Freedom of Info...,01.04.16,Exempt under Section 43 of the Freedom of Info...,Exempt under Section 43 of the Freedom of Info...,NaN,NaN,NaN,Exempt under Section 43 of the Freedom of Info...,Exempt under Section 43 of the Freedom of Info...,2019,Military Capability,NaN,NaN
95,Joint Crypt Key Programme,MOD,exempt under section 26 of freedom of informat...,To support the department's Information Assura...,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,NaN,NaN,NaN,Exempt under Section 26 of Freedom of Informat...,Exempt under Section 26 of Freedom of Informat...,2020,ICT,MOD_0111_1718-Q1,NaN
105,Nuclear Warhead Capability Sustainment Programme,MOD,exempt under section 24 of freedom of informat...,"To deliver and sustain the capability (skills,...",Exempt under Section 24 of Freedom of Informat...,01.04.08,30.04.25,The scheduled baseline project end date at Q2 ...,1088.00,1028.69,20900.75,The 19/2

---

In [ ]:
df1 = pd.read_csv("../data/raw_data/2021_2023.csv", thousands=',')
df2 = pd.read_csv("../data/raw_data/uk_2018-2020.csv", thousands=',')
df3 = pd.read_csv("../data/raw_data/df_2014_2017.csv", thousands=',')

In [ ]:
df = pd.concat([df1, df2, df3])

df = df.drop("Unnamed: 0", axis=1)


df = df.rename({"TOTAL Baseline Benefits (Łm)": "benefits"}, axis=1)

allowed_values = ['Amber', 'Green', 'Red', 'Amber/red', 'Amber/Green']
df['colour_rating'] = df['colour_rating'].apply(lambda x: x if x in allowed_values else np.nan)
df

In [ ]:
df.colour_rating.unique()

In [ ]:
# Use regex to find non-numeric entries
non_numeric_rows_regex = df[~df['yearly_budget'].str.match(r'^\d+(\.\d+)?$', na=False)]

# Display the non-numeric rows
non_numeric_rows_regex["wlc_baseline_incl_NCG"].unique()

In [ ]:
df.info()

In [ ]:
df.info()

In [ ]:
df['yearly_budget'] = pd.to_numeric(df['yearly_budget'], errors='coerce')

In [ ]:
df = pd.read_csv("../data/EDA/dataset_EDA.csv")
df = df.drop("Unnamed: 0", axis=1)
df = df.rename({"TOTAL Baseline Benefits (Łm)": "benefits"}, axis=1)
df['yearly_budget'] = df['yearly_budget'].str.replace('Ł', '', regex=False)
df['yearly_forecast'] = df['yearly_forecast'].str.replace('Ł', '', regex=False)
df['wlc_baseline_incl_NCG'] = df['wlc_baseline_incl_NCG'].str.replace('Ł', '', regex=False)

allowed_values = ['Amber', 'Green', 'Red', 'Amber/red', 'Amber/Green']
df['colour_rating'] = df['colour_rating'].apply(lambda x: x if x in allowed_values else np.nan)

df['yearly_budget'] = pd.to_numeric(df['yearly_budget'], errors='coerce')
df['yearly_forecast'] = pd.to_numeric(df['yearly_forecast'], errors='coerce')
df['wlc_baseline_incl_NCG'] = pd.to_numeric(df['wlc_baseline_incl_NCG'], errors='coerce')

df = df.drop(862)
df = df.drop(1337).reset_index().set_index()

df.head()

In [ ]:
msn.bar(df)

In [ ]:
no_budget = df[df['yearly_budget'].isna()]

In [ ]:
no_budget["year"].value_counts()

In [ ]:
df["year"].value_counts()

In [ ]:
msn.matrix(df)

In [ ]:
# Count unique project names
x = df["yearly_budget"].unique().tolist()
num = 0
for i in x:
    num += 1


In [ ]:
df[df['yearly_budget'].isna()]

In [ ]:

project_no_budget = df[df['yearly_budget'].isna()]["project_name"].unique().tolist()

In [ ]:
project_no_budget

In [ ]:
project_unique = df["project_name"].unique().tolist()

In [ ]:
shared_elements = set(project_unique).intersection(project_no_budget)

In [ ]:
shared_elements

In [ ]:
pd.set_option('display.max_colwidth', None)  # No limit on column width
pd.set_option('display.width', 1000)
len(df[df["project_name"] == project_no_budget[2]]["variance_comment"].index)

In [ ]:
for idx, element in enumerate(project_no_budget):
    if len(df[df["project_name"] == element].index):
        print(df[df["project_name"] == element])

In [ ]:
df[df["project_name"].isin(project_no_budget)]

In [ ]:
# Initialize an empty DataFrame to store the results
filtered_df = pd.DataFrame()

# Loop through the list and filter the DataFrame
for idx, element in enumerate(project_no_budget):
    temp_df = df[df["project_name"] == element]
    if len(temp_df.index) >= 2:  # Ensures you only add if there are at least two rows
        filtered_df = pd.concat([filtered_df, temp_df])

# The filtered_df now contains all the rows from your original DataFrame where
# the project_name is in the project_no_budget list and appears at least twice.
filtered_df.reset_index(drop=True, inplace=True)  # Optional: Reset index if needed

# Now, you can use filtered_df as your resulting DataFrame
filtered_df

In [ ]:
df[df["project_name"]== "Mechanised Infantry Programme"]

In [ ]:
df["project_name"].value_counts().to_list()

In [ ]:
df_emna = pd.read_csv("../data/raw_data/df_2014_2017.csv")

In [ ]:
df